In [3]:
import pytesseract
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import requests
import os
from pdf2image import convert_from_path, convert_from_bytes

from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)

# first level of shortening
import spacy as spacy

from spacy.tokens import Span
from spacy.matcher import Matcher

In [4]:
fName = r"F:\Python\OCR_diploma\docs_base\StauffInvoices\92951827.pdf"
strSeparator = '\n#########################################################\n'

In [5]:
def pdfDocToString(f_name):
    size = 2400
    pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
    f_pdf = open(f_name, 'rb').read()
    images = convert_from_bytes(f_pdf, dpi = 300, size=size)
    text = ''
    for img in images:
        text += pytesseract.image_to_string(img , lang='rus+eng', config='--psm 4' )
    return text

In [6]:
def shortening(text):
    nlp = spacy.blank("en")

    doc = nlp(text)

    matcher = Matcher(nlp.vocab)

    pattern3 = [
        {'LOWER':'CONTRIBUTING'.lower()},
        {'LOWER':'TO'.lower()}, 
        {'LOWER':'YOUR'.lower()}, 
        {'LOWER':'SUCCESS'.lower()}
        ]
    matcher.add('ENDPAGE', [pattern3])

    pattern4 = [
        {'LOWER':'article'},
        {'LOWER':'code'}, 
        {'LOWER':'price'}, 
        {'LOWER':'eur'},
        {'LOWER':'unit'},
        {'LOWER':'amount'},
        {'LOWER':'eur'}
    ]

    pattern4_ = [{'LOWER':'russische'}, {'LOWER':'foed'}]
    matcher.add('STARTPAGE', [pattern4, pattern4_])

    pattern5  = [
        {'LOWER':'doc.no'},
        {'LOWER':'.'},
        {'LOWER':'/'},
        {'LOWER':'date'},
        {'LOWER':'page'}
    ]
    
    pattern6  = [
        {'LOWER':'doc.по'},
        {'LOWER':'.'},
        {'LOWER':'/'},
        {'LOWER':'date'},
        {'LOWER':'page'}
    ] 
    matcher.add('ABOUTNOPAGE', [pattern5, pattern6])

    doc = nlp(text)

    matches = matcher(doc)
    spans = [Span(doc, start, end, label=match_id) for match_id, start, end in matches]

    paging = []

    pageinfo = {'ABOUTNOPAGE':0, 'STARTPAGE':0, 'ENDPAGE':0}
    shortenText = ''
    firstPage = True
    txt = '''import spacy
    from spacy.tokens import Span

    nlp = spacy.blank("en")\n'''
    docs  = []
    npos = 0
    npage = 0
    for sp in spans:
        if sp.label_  == 'ABOUTNOPAGE':
            pageinfo[sp.label_] = sp.end
        elif sp.label_  == 'STARTPAGE':
            pageinfo[sp.label_] = sp.end
        elif sp.label_  == 'ENDPAGE':
            pageinfo[sp.label_] = sp.start
            paging.append(pageinfo)
            
            if firstPage ==True:
                shortenText = doc[:pageinfo['ENDPAGE']].text
                firstPage = False
                docs.append(doc[:pageinfo['ENDPAGE']].text)
                txt = txt + strSeparator + doc[:pageinfo['ENDPAGE']].text 
            else:
                shortenText = shortenText +  doc[pageinfo['STARTPAGE']:pageinfo['ENDPAGE']].text
                docs.append(doc[pageinfo['STARTPAGE']:pageinfo['ENDPAGE']].text)
                print(pageinfo, pageinfo['ENDPAGE'] - pageinfo['STARTPAGE'])
                txt = txt + strSeparator + doc[pageinfo['STARTPAGE']:pageinfo['ENDPAGE']].text 

                
                #break
            pageinfo = {'ABOUTNOPAGE':0, 'STARTPAGE':0, 'ENDPAGE':0}
            npage += 1

        #print(pageinfo)
    #print(txt)
    print(len(paging))
    return shortenText

#with open(fName.replace('.txt', 's.txt'), encoding="utf8", mode='w') as f:
#    f.write(txt)


In [19]:
catName = fName.split('\\')[-1].split('.')[0]
print('\\'.join(fName.split('\\')[:-1]))
if os.path.isdir(catName):
    print('Dir is')

F:\Python\OCR_diploma\docs_base\StauffInvoices


In [ ]:
print(txtS)

In [7]:

text = pdfDocToString(f_name=fName)
txtS = shortening(text)

print(txtS)

{'ABOUTNOPAGE': 560, 'STARTPAGE': 595, 'ENDPAGE': 837} 242
{'ABOUTNOPAGE': 1064, 'STARTPAGE': 1099, 'ENDPAGE': 1314} 215
{'ABOUTNOPAGE': 1541, 'STARTPAGE': 1576, 'ENDPAGE': 1810} 234
{'ABOUTNOPAGE': 2037, 'STARTPAGE': 2074, 'ENDPAGE': 2285} 211
{'ABOUTNOPAGE': 2512, 'STARTPAGE': 2549, 'ENDPAGE': 2765} 216
{'ABOUTNOPAGE': 2992, 'STARTPAGE': 3027, 'ENDPAGE': 3250} 223
{'ABOUTNOPAGE': 3477, 'STARTPAGE': 3512, 'ENDPAGE': 3726} 214
{'ABOUTNOPAGE': 3953, 'STARTPAGE': 3988, 'ENDPAGE': 4219} 231
{'ABOUTNOPAGE': 4446, 'STARTPAGE': 4483, 'ENDPAGE': 4708} 225
{'ABOUTNOPAGE': 4935, 'STARTPAGE': 4970, 'ENDPAGE': 5186} 216
{'ABOUTNOPAGE': 5413, 'STARTPAGE': 5450, 'ENDPAGE': 5680} 230
{'ABOUTNOPAGE': 5907, 'STARTPAGE': 5944, 'ENDPAGE': 6168} 224
{'ABOUTNOPAGE': 6395, 'STARTPAGE': 6432, 'ENDPAGE': 6671} 239
{'ABOUTNOPAGE': 6898, 'STARTPAGE': 6935, 'ENDPAGE': 7163} 228
{'ABOUTNOPAGE': 7390, 'STARTPAGE': 7427, 'ENDPAGE': 7649} 222
{'ABOUTNOPAGE': 7876, 'STARTPAGE': 7911, 'ENDPAGE': 8125} 214
{'ABOUTNOPA

In [29]:
def text2spans(rtest):
    rtest = rtest.replace(strSeparator, '')

    nlp = spacy.blank("en")
    matcher = Matcher(nlp.vocab)

    patternPurchase = [{'LOWER': 'purchase'}, 
        {'LOWER':'order'}, 
        {'LOWER':'number'}, 
        {'LOWER': ':'}]
    matcher.add('PTRPURCHASE', [patternPurchase])

    patternTariff = [{'LOWER': 'customs'}, 
        {'LOWER':'tariff'}, 
        {'IS_ALPHA': True},
        {'IS_PUNCT': True, 'OP': '*'}, 
        {'IS_DIGIT': True}]
    matcher.add('PTRTARIFF', [patternTariff])

    patternCountry = [{'LOWER': 'country'}, 
        {'LOWER':'of'} , 
        {'LOWER': 'origin'}, 
        {'LOWER': ':'}, 
        {'IS_ALPHA': True}]
    matcher.add('PTRCOUNTRY', [patternCountry])

    patternPC = [{'LIKE_NUM': True}, {'LIKE_NUM': True},
        {'LOWER' : {'in' : ['pc', 'рс']}},
        {'LIKE_NUM':True}    ] 

    matcher.add('PTROFPC', [patternPC])

    doc1 = nlp(rtest)

    #print("Verbs:", [token.text for token in doc1[45:54]])

    matches = matcher(doc1)
    spans = [Span(doc1, start, end, label=match_id) for match_id, start, end in matches]

    startSpan = 0
    endSpan = 0
    #posTariff = 0
    #txt2 = 'import json \nimport spacy \nfrom spacy.matcher import Matcher \nfrom spacy.tokens import Span, DocBin \ndocs = [] \n'
    #nPos = 0
    docs = []
    pos2PC = 0
    for sp in spans:
        if sp.label_  == 'PTRPURCHASE':
            startSpan = sp.start
            
        #elif sp.label_  == 'PTRTARIFF':             posTariff = sp.end-1
            
        #elif sp.label_  == 'PTROFPC':             pos2PC = sp.start
        
        elif sp.label_  == 'PTRCOUNTRY':
            endSpan = sp.end
            #pos2PC = pos2PC - startSpan  
            

            docs.append(doc1[startSpan:endSpan].text)
            
    return docs

In [30]:
docs = text2spans(txtS)

In [12]:
type(docs[1])

str

In [31]:
nlp_ = spacy.load(r"C:\Users\AdminAsus\source\python\spacyST\spacySt-1\output\model-best")
for doc in docs:
    docx = nlp_(doc)
    print(docx.ents)
    #for ent in docx.ents:
    #    print(ent.text, ent.label_)

(SR-1-06, 1454, 100, 6, 1020014049, 13,59, 1, 81,54, 84219990, Germany)
(SR-1-06, 1484, 3900, 1, 6020003319, 1.293,34, 1, 1.293,34, 84662020, Germany)
(SR-1-06, 1505, 4000, 40, 6100035587, 30,18, 1, 1.207,20, 90262040, Germany)
(SR-1-06, 1505, 4100, 85, 1910020866, 30,18, 1, 2.565,30, 90262040, Germany)
(SR-1-06, 1505, 4200, 20, 6100075627, 33,03, 1, 660,60, 90262040, Poland)
(SR-1-06, 1726, 4300, 28, 6100098810, 18,17, 1, 508,76, 73072390, Italy)
(SR-1-06, 1799, 4400, 50, 1120002537, 38,09, 100, 19,05, 73269098, Germany)
(SR-1-06, 1824, 4500, 5.000, 6010001858, 58,36, 100, 2.918,00, 73079910, Germany)
(SR-1-06, 1840, 4600, 15.000, 6100145133, 0,23, 1, 3.450,00, 73182100, Origin)
(SR-1-06, 1858, 4700, 50, 1120002537, 38,09, 100, 19,05, 73269098, Germany)
(SR-1-06, 1872, 4800, 25, 1120002537, 38,09, 100, 9,52, 73269098, Germany)
(SR-1-06, 1873, 4900, 12, 6100160247, 85,90, 1, 1.030,80, 84812010, Germany)
(SR-1-06, 1878, 5000, 50, 1120002537, 38,09, 100, 19,05, 73269098, Germany)
(SR-1-0